In [2]:
# import sys
import os
import gc
import warnings
import random
from copy import deepcopy
import random
import math

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch.utils.data import Dataset, DataLoader
from madgrad import MADGRAD, MirrorMADGRAD
from torch.optim import AdamW
from sklearn.metrics import f1_score
from transformers import (
    get_constant_schedule,
    get_constant_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
    get_cosine_with_hard_restarts_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_polynomial_decay_schedule_with_warmup
)

from sklearn.model_selection import StratifiedKFold,KFold
from tqdm import tqdm
import cv2
from PIL import Image
import albumentations as A
from sklearn.utils.class_weight import compute_class_weight
from transformers import AutoFeatureExtractor, AutoModelForImageClassification

warnings.filterwarnings("ignore")
tqdm.pandas()

c:\users\len\pythonproject1\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transforms=torchvision.transforms.Compose([
    torchvision.transforms.RandomRotation(45),
    torchvision.transforms.RandomHorizontalFlip(p=0.6),
    torchvision.transforms.RandomVerticalFlip(p=0.6),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize(size=(256,256)),
    torchvision.transforms.Lambda(lambda a: a / 255),
    
#     torchvision.transforms.Normalize(mean, std)
])

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(f"./data_ext/train", transform=transforms)

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, weight, gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
        self.ce = torch.nn.CrossEntropyLoss(weight=weight)

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

In [ ]:
class ResNet(nn.Module):
    def __init__(self, backbone, output_dim):
        super().__init__()
        self.backbone = backbone
        del backbone.fc
        self.backbone.dropout = nn.Dropout(p=0.1)
        self.backbone.fc = torch.nn.Linear(in_features=2048, out_features=output_dim, bias=True)
        
    def forward(self, image):
        out = self.backbone(image)
#         out = self.out(x)
        return out

In [ ]:
def train_epoch(model, data_loader, loss_function, optimizer, scheduler, device, n_accumulated_grads=0):
    model.to(device)
    model.train()
    total_train_loss = 0

    dl_size = len(data_loader)
    
    preds = []
    targets = []

    batch_i = 0
    steps_to_accumulate_grads = 0
    for batch in tqdm(data_loader):
        image, target = batch
        image = image.to(device)
        target = target.to(device)
        
        optimizer.zero_grad()        
        logits = model(image)
        
        preds.append(logits.argmax(dim=1))
        targets.append(target)
                
        loss = loss_function(logits, target)
        total_train_loss += loss.item()
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        if steps_to_accumulate_grads == n_accumulated_grads:
            optimizer.step()
            scheduler.step()
            steps_to_accumulate_grads = 0
        else:
            steps_to_accumulate_grads += 1
            
    if steps_to_accumulate_grads != 0:
        optimizer.step()
        scheduler.step()
    
    preds = torch.cat(preds, dim=0)
    targets = torch.cat(targets, dim=0)
    acc = (targets == preds).sum() / preds.shape[0]
    f1 = f1_score(preds.cpu(), targets.cpu(), average='macro')
    
    metrics = {
        "Train Loss": total_train_loss / dl_size,
        "Train Accuracy": acc.item(),
        "Train F1": f1.item()
    }
    
    
    return metrics
    
    
def eval_epoch(model, data_loader, loss_function, device):
    model.to(device)
    model.eval()
    total_train_loss = 0
    
    preds = []
    targets = []

    dl_size = len(data_loader)

    
    for batch in tqdm(data_loader):
        image, target = batch
        image = image.to(device)
        target = target.to(device)
        
        with torch.no_grad():
            logits = model(image)
            preds.append(logits.argmax(dim=1))
            targets.append(target)
        
        loss = loss_function(logits, target)
        total_train_loss += loss.item()
    
    preds = torch.cat(preds, dim=0)
    targets = torch.cat(targets, dim=0)
    acc = (targets == preds).sum() / preds.shape[0]
    f1 = f1_score(preds.cpu(), targets.cpu(), average='macro')
    
    metrics = {
        "Eval Loss": total_train_loss / dl_size,
        "Eval Accuracy": acc.item(),
        "Eval F1": f1.item()
    }
    
    return metrics

In [ ]:
def cross_validation(project_name,
                     model, 
                     dataset, 
                     loss_function, 
                     strat_array=None,
                     device=torch.device("cuda"),
                     random_state: int=69, 
                     shuffle: bool=True, 
                     n_folds: int=4, 
                     epochs: int=5, 
                     lr: float=1e-6,
                     start_fold: int=0, 
                     batch_size: int=32,
                     iters_to_accumulate=None,
                     n_accumulated_grads: int = 0):
    random.seed(random_state),
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    torch.cuda.manual_seed_all(random_state)
    
    loss_function.to(device)
    if strat_array:
        kfold = StratifiedKFold(n_folds, shuffle=shuffle, random_state=random_state)
        split = kfold.split(dataset, strat_array)
    else: 
        kfold = KFold(n_folds, shuffle=shuffle, random_state=random_state)
        split = kfold.split(dataset)

    for fold, (train_ids, eval_ids) in enumerate(split):
        if fold >= start_fold:
            print(f'FOLD {fold}')
            print('--------------------------------')
            
            
            '''run = wandb.init(
                name=f"fold_{fold}",
                project=f"{project_name}_fold_{fold}",
                config={ 
                         "random_state": random_state, 
                         "shuffle": shuffle,
                         "epochs": epochs, 
                         "learning_rate": lr,
                         "batch_size": batch_size,
                         "iters_to_accumulate": iters_to_accumulate
                        }
            )'''

            optimizer = MADGRAD(
            model.parameters(),
            lr = lr, # args.learning_rate - default is 5e-5, our notebook had 2e-5
        )

            train_subsampler = torch.utils.data.Subset(dataset,  train_ids)
            train_loader = torch.utils.data.DataLoader(
                          train_subsampler, 
                          batch_size=batch_size,
                          shuffle=shuffle)

            eval_subsampler = torch.utils.data.Subset(dataset,  eval_ids)
            eval_loader = torch.utils.data.DataLoader(
                          eval_subsampler,
                          batch_size=batch_size,
                          shuffle=shuffle)
            
            total_steps = len(train_loader) * epochs 

            scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps = 0, # Default value in run_glue.py
                                                    num_training_steps = total_steps)

            mrrs = []

            for epoch_i in range(0, epochs):
                train_metrics = train_epoch(model, train_loader, loss_function, optimizer, scheduler, device)
                eval_metrics = eval_epoch(model, eval_loader, loss_function, device)
                
                print(f"EPOCH: {epoch_i}")
                print(train_metrics)
                print(eval_metrics)
                
                #run.log(train_metrics)
                #run.log(eval_metrics)
                            
            #run.finish()


def single_model(model, 
                     dataset, 
                     loss_function, 
                     device=torch.device("cuda"),
                     random_state: int=69, 
                     shuffle=True,
                     epochs: int=15, 
                     lr: float=1e-6,
                     batch_size: int=32,
                     start_epoch=0
                     ):
    random.seed(random_state),
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    torch.cuda.manual_seed_all(random_state)
    
    loss_function.to(device)
    model.to(device)

    optimizer = MADGRAD(
        model.parameters(),
        lr = lr, # args.learning_rate - default is 5e-5, our notebook had 2e-5
    )
    
    fold_path = f'saves/resnet'
    if not os.path.exists(fold_path):
        os.mkdir(fold_path)
        
    data_loader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=batch_size,
                    shuffle=shuffle
    )
    
    total_steps = len(data_loader) * epochs 

    scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

    for epoch_i in range(0, epochs):
        if epoch_i >= start_epoch:
            train_metrics = train_epoch(model, data_loader, loss_function, optimizer, scheduler, device)
            epoch_path = fold_path+f'/epoch_{epoch_i}'
            if not os.path.exists(epoch_path):
                os.mkdir(epoch_path)
            save_model(model, epoch_path)
            print("EPOCH", epoch_i)
            print(train_metrics)
            # eval_epoch(fold_model, eval_loader, loss_function, device)

In [ ]:
def save_model(model, path):
    torch.save(model, path+'/model.pt')

In [ ]:
def cross_validation(project_name,
                     model, 
                     dataset, 
                     loss_function, 
                     strat_array=None,
                     device=torch.device("cuda"),
                     random_state: int=69, 
                     shuffle: bool=True, 
                     n_folds: int=4, 
                     epochs: int=5, 
                     lr: float=1e-6,
                     start_fold: int=0, 
                     batch_size: int=32,
                     iters_to_accumulate=None,
                     n_accumulated_grads: int = 0):
    random.seed(random_state),
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    torch.cuda.manual_seed_all(random_state)
    
    loss_function.to(device)
    if strat_array:
        kfold = StratifiedKFold(n_folds, shuffle=shuffle, random_state=random_state)
        split = kfold.split(dataset, strat_array)
    else: 
        kfold = KFold(n_folds, shuffle=shuffle, random_state=random_state)
        split = kfold.split(dataset)

    for fold, (train_ids, eval_ids) in enumerate(split):
        if fold >= start_fold:
            print(f'FOLD {fold}')
            print('--------------------------------')
            
            
            '''run = wandb.init(
                name=f"fold_{fold}",
                project=f"{project_name}_fold_{fold}",
                config={ 
                         "random_state": random_state, 
                         "shuffle": shuffle,
                         "epochs": epochs, 
                         "learning_rate": lr,
                         "batch_size": batch_size,
                         "iters_to_accumulate": iters_to_accumulate
                        }
            )'''

            optimizer = MADGRAD(
            model.parameters(),
            lr = lr, # args.learning_rate - default is 5e-5, our notebook had 2e-5
        )

            train_subsampler = torch.utils.data.Subset(dataset,  train_ids)
            train_loader = torch.utils.data.DataLoader(
                          train_subsampler, 
                          batch_size=batch_size,
                          shuffle=shuffle)

            eval_subsampler = torch.utils.data.Subset(dataset,  eval_ids)
            eval_loader = torch.utils.data.DataLoader(
                          eval_subsampler,
                          batch_size=batch_size,
                          shuffle=shuffle)
            
            total_steps = len(train_loader) * epochs 

            scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps = 0, # Default value in run_glue.py
                                                    num_training_steps = total_steps)

            mrrs = []

            for epoch_i in range(0, epochs):
                train_metrics = train_epoch(model, train_loader, loss_function, optimizer, scheduler, device)
                eval_metrics = eval_epoch(model, eval_loader, loss_function, device)
                
                print(f"EPOCH: {epoch_i}")
                print(train_metrics)
                print(eval_metrics)
                
                #run.log(train_metrics)
                #run.log(eval_metrics)
                            
            #run.finish()


def single_model(model, 
                     dataset, 
                     loss_function, 
                     device=torch.device("cuda"),
                     random_state: int=69, 
                     shuffle=True,
                     epochs: int=15, 
                     lr: float=1e-6,
                     batch_size: int=32,
                     start_epoch=0
                     ):
    random.seed(random_state),
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    torch.cuda.manual_seed_all(random_state)
    
    loss_function.to(device)
    model.to(device)

    optimizer = MADGRAD(
        model.parameters(),
        lr = lr, # args.learning_rate - default is 5e-5, our notebook had 2e-5
    )
    
    fold_path = f'saves/resnet'
    if not os.path.exists(fold_path):
        os.mkdir(fold_path)
        
    data_loader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=batch_size,
                    shuffle=shuffle
    )
    
    total_steps = len(data_loader) * epochs 

    scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

    for epoch_i in range(0, epochs):
        if epoch_i >= start_epoch:
            train_metrics = train_epoch(model, data_loader, loss_function, optimizer, scheduler, device)
            epoch_path = fold_path+f'/epoch_{epoch_i}'
            if not os.path.exists(epoch_path):
                os.mkdir(epoch_path)
            save_model(model, epoch_path)
            print("EPOCH", epoch_i)
            print(train_metrics)
            # eval_epoch(fold_model, eval_loader, loss_function, device)

In [ ]:
strat_array = []
target_class = 0
for folder in tqdm(os.listdir('data_ext/train')):
    for _ in range(len(os.listdir('data_ext/train'+f'/{folder}'))):
        strat_array.append(target_class)
    print(folder)
    target_class += 1

In [ ]:
loss_function=FocalLoss(weight=torch.tensor(correct_weights).float())

# ResNet50 training

In [3]:
path_to_pretrain_resnet50 = 'model/model.pt'
resnet_weights = torch.load(path_to_pretrain_resnet50)
backbone = torchvision.models.resnet50(weights = resnet_weights)
effnet_best = ResNet(backbone, 11)

FileNotFoundError: [Errno 2] No such file or directory: 'model/model.pt'

In [ ]:
single_model(model = effnet_best, 
            dataset = train_dataset, 
            loss_function = loss_function, 
            device = torch.device("cuda"),
            random_state = 69, 
            shuffle = True,
            epochs = 24, 
            lr = 1e-5,
            batch_size = 16)

In [ ]:
path_to_pretrain_resnet50 = 'model/model.pt'
resnet_weights = torch.load(path_to_pretrain_resnet50)
backbone = torchvision.models.resnet50(weights = resnet_weights)
effnet_rs_30 = ResNet(backbone, 11)

In [ ]:
single_model(model = effnet_rs_30, 
            dataset = train_dataset, 
            loss_function = loss_function, 
            device = torch.device("cuda"),
            random_state = 30, 
            shuffle = True,
            epochs = 24, 
            lr = 1e-5,
            batch_size = 16)

# EfficientNet_b1

In [ ]:
model_weights = torchvision.models.EfficientNet_B1_Weights.IMAGENET1K_V1
effnet_b1 = torchvision.models.efficientnet_b1(weights=model_weights)
effnet_b1.classifier[1] = nn.Linear(in_features = 1536, out_features =11, bias = True) 

In [ ]:
single_model(model = effnet_b1, 
            dataset = train_dataset, 
            loss_function = loss_function, 
            device = torch.device("cuda"),
            random_state = 69, 
            shuffle = True,
            epochs = 40, 
            lr = 1e-5,
            batch_size = 16)

In [ ]:
model_weights = torchvision.models.EfficientNet_B1_Weights.IMAGENET1K_V1
effnet_b1_rs_40 = torchvision.models.efficientnet_b1(weights=model_weights)
effnet_b1_rs_40.classifier[1] = nn.Linear(in_features = 1536, out_features =11, bias = True) 

In [ ]:
single_model(model = effnet_b1_rs_40, 
            dataset = train_dataset, 
            loss_function = loss_function, 
            device = torch.device("cuda"),
            random_state = 40, 
            shuffle = True,
            epochs = 40, 
            lr = 1e-5,
            batch_size = 16)

# EfficientNet b3

In [ ]:
model_weights = torchvision.models.EfficientNet_B3_Weights.IMAGENET1K_V1
effnet_best = torchvision.models.efficientnet_b3(weights=model_weights)
effnet_best.classifier[1] = nn.Linear(in_features = 1536, out_features =11, bias = True) 

In [ ]:
single_model(model = effnet_best, 
            dataset = train_dataset, 
            loss_function = loss_function, 
            device = torch.device("cuda"),
            random_state = 69, 
            shuffle = True,
            epochs = 35, 
            lr = 1e-5,
            batch_size = 16)

In [ ]:
model_weights = torchvision.models.EfficientNet_B3_Weights.IMAGENET1K_V1
effnet_rs_30 = torchvision.models.efficientnet_b3(weights=model_weights)
effnet_rs_30.classifier[1] = nn.Linear(in_features = 1536, out_features =11, bias = True) 

In [ ]:
single_model(model = effnet_best, 
            dataset = train_dataset, 
            loss_function = loss_function, 
            device = torch.device("cuda"),
            random_state = 30, 
            shuffle = True,
            epochs = 35, 
            lr = 1e-5,
            batch_size = 16)

# VIT

In [ ]:
def train_epoch_tr(model, data_loader, loss_function, optimizer, scheduler, device, n_accumulated_grads=0):
    model.to(device)
    model.train()
    total_train_loss = 0

    dl_size = len(data_loader)
    
    preds = []
    targets = []

    batch_i = 0
    steps_to_accumulate_grads = 0
    for batch in tqdm(data_loader):
        image, target = batch
        image = image["pixel_values"][0].to(device)
        target = target.to(device)
        
        optimizer.zero_grad()        
        logits = model(image).logits
                
        preds.append(logits.argmax(dim=1))
        targets.append(target)
                
        loss = loss_function(logits, target)
        total_train_loss += loss.item()
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    preds = torch.cat(preds, dim=0)
    targets = torch.cat(targets, dim=0)
    acc = (targets == preds).sum() / preds.shape[0]
    f1 = f1_score(preds.cpu(), targets.cpu(), average='macro')
    
    metrics = {
        "Train Loss": total_train_loss / dl_size,
        "Train Accuracy": acc.item(),
        "Train F1": f1.item()
    }
    
    
    return metrics
    
    
def eval_epoch_tr(model, data_loader, loss_function, device):
    model.to(device)
    model.eval()
    total_train_loss = 0
    
    preds = []
    targets = []

    dl_size = len(data_loader)

    
    for batch in tqdm(data_loader):
        image, target = batch
        image = image["pixel_values"][0].to(device)
        target = target.to(device)
        
        with torch.no_grad():
            logits = model(image).logits
            preds.append(logits.argmax(dim=1))
            targets.append(target)
        
        loss = loss_function(logits, target)
        total_train_loss += loss.item()
    
    preds = torch.cat(preds, dim=0)
    targets = torch.cat(targets, dim=0)
    acc = (targets == preds).sum() / preds.shape[0]
    f1 = f1_score(preds.cpu(), targets.cpu(), average='macro')
    
    metrics = {
        "Eval Loss": total_train_loss / dl_size,
        "Eval Accuracy": acc.item(),
        "Eval F1": f1.item()
    }
    
    return metrics

In [ ]:
def cross_validation(project_name,
                     model, 
                     dataset, 
                     loss_function, 
                     strat_array=None,
                     device=torch.device("cuda"),
                     random_state: int=69, 
                     shuffle: bool=True, 
                     n_folds: int=4, 
                     epochs: int=5, 
                     lr: float=1e-6,
                     start_fold: int=0, 
                     batch_size: int=32,
                     iters_to_accumulate=None,
                     n_accumulated_grads: int = 0):
    random.seed(random_state),
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    torch.cuda.manual_seed_all(random_state)
    
    loss_function.to(device)
    if strat_array:
        kfold = StratifiedKFold(n_folds, shuffle=shuffle, random_state=random_state)
        split = kfold.split(dataset, strat_array)
    else: 
        kfold = KFold(n_folds, shuffle=shuffle, random_state=random_state)
        split = kfold.split(dataset)

    for fold, (train_ids, eval_ids) in enumerate(split):
        if fold >= start_fold:
            print(f'FOLD {fold}')
            print('--------------------------------')
            
            
            '''run = wandb.init(
                name=f"fold_{fold}",
                project=f"{project_name}_fold_{fold}",
                config={ 
                         "random_state": random_state, 
                         "shuffle": shuffle,
                         "epochs": epochs, 
                         "learning_rate": lr,
                         "batch_size": batch_size,
                         "iters_to_accumulate": iters_to_accumulate
                        }
            )'''

            optimizer = MADGRAD(
            model.parameters(),
            lr = lr, # args.learning_rate - default is 5e-5, our notebook had 2e-5
        )

            train_subsampler = torch.utils.data.Subset(dataset,  train_ids)
            train_loader = torch.utils.data.DataLoader(
                          train_subsampler, 
                          batch_size=batch_size,
                          shuffle=shuffle)

            eval_subsampler = torch.utils.data.Subset(dataset,  eval_ids)
            eval_loader = torch.utils.data.DataLoader(
                          eval_subsampler,
                          batch_size=batch_size,
                          shuffle=shuffle)
            
            total_steps = len(train_loader) * epochs 

            scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps = 0, # Default value in run_glue.py
                                                    num_training_steps = total_steps)

            mrrs = []

            for epoch_i in range(0, epochs):
                train_metrics = train_epoch_tr(model, train_loader, loss_function, optimizer, scheduler, device)
                eval_metrics = eval_epoch_tr(model, eval_loader, loss_function, device)
                
                print(f"EPOCH: {epoch_i}")
                print(train_metrics)
                print(eval_metrics)
                
                #run.log(train_metrics)
                #run.log(eval_metrics)
                            
            #run.finish()


def single_model_tr(model, 
                     dataset, 
                     loss_function, 
                     device=torch.device("cuda"),
                     random_state: int=69, 
                     shuffle=True,
                     epochs: int=15, 
                     lr: float=1e-6,
                     batch_size: int=32,
                     start_epoch=0
                     ):
    random.seed(random_state),
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    torch.cuda.manual_seed_all(random_state)
    
    loss_function.to(device)
    model.to(device)

    optimizer = MADGRAD(
        model.parameters(),
        lr = lr, # args.learning_rate - default is 5e-5, our notebook had 2e-5
    )
    
    fold_path = f'vit'
    if not os.path.exists(fold_path):
        os.mkdir(fold_path)
        
    data_loader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=batch_size,
                    shuffle=shuffle
    )
    
    total_steps = len(data_loader) * epochs 

    scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

    for epoch_i in range(0, epochs):
        if epoch_i >= start_epoch:
            train_metrics = train_epoch_tr(model, data_loader, loss_function, optimizer, scheduler, device)
            epoch_path = fold_path+f'/epoch_{epoch_i + 9}'
            if not os.path.exists(epoch_path):
                os.mkdir(epoch_path)
            save_model(model, epoch_path)
            print("EPOCH", epoch_i)
            print(train_metrics)
            # eval_epoch(fold_model, eval_loader, loss_function, device)

In [ ]:
extractor = AutoFeatureExtractor.from_pretrained("therealcyberlord/stanford-car-vit-patch16")
transformer = AutoModelForImageClassification.from_pretrained("therealcyberlord/stanford-car-vit-patch16")
transformer.classifier = nn.Linear(768, 11)

In [ ]:
single_model_tr(model = transformer, 
             dataset = train_dataset, 
             loss_function = loss_function, 
             device = torch.device("cuda"),
             random_state = 69, 
             shuffle = True,
             epochs = 15, 
             lr = 1e-5,
             batch_size = 16)

In [ ]:
extractor = AutoFeatureExtractor.from_pretrained("therealcyberlord/stanford-car-vit-patch16")
transformer_1337 = AutoModelForImageClassification.from_pretrained("therealcyberlord/stanford-car-vit-patch16")
transformer_1337.classifier = nn.Linear(768, 11)

In [ ]:
single_model_tr(model = transformer_1337, 
             dataset = train_dataset, 
             loss_function = loss_function, 
             device = torch.device("cuda"),
             random_state = 1337, 
             shuffle = True,
             epochs = 15, 
             lr = 1e-5,
             batch_size = 16)

In [ ]:
extractor = AutoFeatureExtractor.from_pretrained("therealcyberlord/stanford-car-vit-patch16")
transformer_666 = AutoModelForImageClassification.from_pretrained("therealcyberlord/stanford-car-vit-patch16")
transformer_666.classifier = nn.Linear(768, 11)

In [ ]:
single_model_tr(model = transformer_666, 
             dataset = train_dataset, 
             loss_function = loss_function, 
             device = torch.device("cuda"),
             random_state = 666, 
             shuffle = True,
             epochs = 15, 
             lr = 1e-5,
             batch_size = 16)

# Predict

In [ ]:
models = [
    resnet_best, 
    resnet_rs_30,
    effnet_best,
    effnet_rs_30,
    effnet_b1,
    effnet_b1_rs_40,
]
transformers = [
    transformer,
    transformer_1337,
    transformer_666,
]

In [ ]:
test_transforms=torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize(size=(256,256)),
    torchvision.transforms.Lambda(lambda a: a / 255),
])

transformer_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize(size=(224,224)),
    torchvision.transforms.Lambda(lambda a: extractor(a)),
])

In [ ]:
def predict_blending(models, transformers, root, device, test_transform, trsf_transforms):
    for model in models:
        model.to(device)
        model.eval()
    
    transformer.eval()
    transformer.to(device)
    preds = []
    img_files = sorted(os.listdir(root), key=lambda x: int(x.split(".")[0]))
    print(img_files)
    for img_file in tqdm(img_files):
        img_rgb = Image.open(root + img_file)
        image = test_transform(img_rgb).to(device)
        image_trsf = trsf_transforms(img_rgb)
        image_trsf = torch.tensor(image_trsf["pixel_values"][0]).unsqueeze(dim=0).to(device)
        pred = torch.zeros(1, 11)
        with torch.no_grad():
            for model in models:
                pred += nn.Softmax(dim = 1)(model(image.unsqueeze(dim=0)).cpu())
            for trsf in transformers:
                pred += nn.Softmax(dim = 1)(trsf(image_trsf).logits.cpu())
            pred /= len(models) + len(transformers)
            preds.append(pred.argmax(dim = 1).item())
    return pd.Series(preds)

In [ ]:
pred_blend = predict_blending(models = models, transformers = transformers, device = 'cuda',
                              root ='data_ext/public_test/', test_transform = test_transforms, 
                              trsf_transforms=transformer_transforms)

In [ ]:
pred_blend = pred_blend.apply(lambda x: train_dataset.classes[x])

In [ ]:
pred_blend.to_csv("final_submission.csv", index=False)